In [1]:
import sys
sys.path.append("../src")

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib notebook

# Load dataset and do some preprocessing

In [3]:
train_path = "../data/train.csv"

In [4]:
df_data = pd.read_csv(train_path, sep=",")

In [5]:
df_data.head()

,full_label,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,CFH,FUCA2,GCLC,NFYA,...,AC069544.2,AC006460.2,AL356275.1,AC092329.4,MIR205,AC066615.1,AC105233.5,AC073111.5,AC079781.5,label
0,369_M01_CTRL,8.869526,5.648693,7.664066,8.370360,6.257695,5.270360,7.475895,8.269031,8.279492,...,5.319481,5.045370,5.648693,6.235458,4.523640,3.984119,4.523640,5.218771,5.319481,0
1,369_M05_CTRL,8.705657,5.932060,7.820206,8.362421,6.140431,5.573240,7.791254,8.201606,8.122618,...,4.741939,4.523022,5.534549,6.014217,4.523022,3.984119,5.105518,5.268960,5.409834,0
2,371_M02_CTRL,8.623002,3.984119,7.419094,8.498542,6.160291,5.588960,8.013386,7.919199,8.520048,...,5.588960,5.146627,4.816944,6.716586,4.816944,3.984119,4.816944,5.146627,5.390389,0
3,371_M07_CTRL,9.434875,5.866242,8.337313,8.139628,7.505509,3.984119,7.329864,8.352620,8.048476,...,3.984119,3.984119,4.650987,5.553634,4.650987,3.984119,4.919178,3.984119,4.650987,0
4,371_M11_CTRL,9.055039,5.240241,7.811833,8.236604,6.829787,4.793247,7.691507,8.441777,8.376590,...,4.968903,4.559957,5.453445,6.161526,4.559957,3.984119,4.559957,5.240241,5.787031,0


In [6]:
df_data.T.describe()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
count,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,...,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000,15106.000000
unique,1413.000000,1436.000000,496.000000,748.000000,847.000000,1983.000000,1577.000000,2321.000000,2821.000000,2768.000000,...,1210.000000,1352.000000,1987.000000,487.000000,863.000000,685.000000,1983.000000,2928.000000,2533.000000,2567.000000
top,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119,...,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119,3.984119
freq,663.000000,632.000000,1534.000000,2424.000000,1074.000000,148.000000,877.000000,167.000000,141.000000,419.000000,...,716.000000,609.000000,632.000000,1433.000000,945.000000,1392.000000,145.000000,131.000000,173.000000,142.000000


In [7]:
df_features = df_data[df_data.columns.difference(['full_label', 'label'])]
df_labels = df_data["label"]

In [8]:
df_features.head()

,A1BG-AS1,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,AAK1,...,ZYG11B,ZYX,ZZEF1,ZZZ3,août-01,août-02,août-03,août-04,janv.-01,snoU2_19
0,4.908175,12.037180,5.045370,5.045370,7.381365,8.263771,5.164377,6.423369,8.350658,8.210091,...,8.616981,6.765996,8.704889,8.976108,8.608716,9.066324,8.570926,8.886709,7.581604,4.366722
1,4.523022,12.272072,5.105518,5.217421,7.398009,8.515874,5.749570,6.255497,8.001646,7.956770,...,8.630280,6.669450,8.827865,9.128019,8.795552,9.201074,8.580553,8.855992,7.463924,3.984119
2,5.758390,6.039895,5.146627,3.984119,7.982674,8.432022,6.991751,5.758390,8.719075,8.237920,...,7.982674,6.466886,8.826486,9.121773,8.541237,9.107652,8.476709,8.602993,8.157784,3.984119
3,3.984119,11.059675,3.984119,3.984119,6.807586,7.532781,7.390862,6.258603,8.993143,8.290386,...,8.827956,3.984119,8.678210,9.346056,8.367766,9.031639,9.114668,8.849671,8.139628,3.984119
4,4.968903,10.311189,4.559957,3.984119,7.458173,8.095056,6.796347,6.161526,8.319912,8.224298,...,8.720871,5.453445,8.473294,9.621207,8.493928,9.181128,8.901494,8.983960,8.308303,3.984119


In [9]:
df_labels.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

## Set X (y) matrix(vector)

In [10]:
X = df_features.values
print(X.shape)

(29, 15104)


In [11]:
y = df_labels.values
print(y.shape)

(29,)


# Logistic regression

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [13]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(21, 15104)
(21,)
(8, 15104)
(8,)


## Define the models

In [14]:
clf_l1_LR = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, multi_class='multinomial')
clf_l2_LR = LogisticRegression(penalty='l2', solver='saga', max_iter=1000, multi_class='multinomial')

## Train models

In [15]:
clf_l1_LR.fit(x_train, y_train)

/Users/sinitame/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l1',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
clf_l2_LR.fit(x_train, y_train)

/Users/sinitame/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

## Test models

In [17]:
predictions_l1 = clf_l1_LR.predict(x_test)

In [18]:
predictions_l2 = clf_l2_LR.predict(x_test)

In [19]:
score_l1 = clf_l1_LR.score(x_test, y_test)
print(score_l1)

0.5


In [20]:
score_l2 = clf_l2_LR.score(x_test, y_test)
print(score_l1)

0.5


## Confusion matrix

In [21]:
cm_l1 = metrics.confusion_matrix(y_test, predictions_l1)
print(cm_l1)

[[0 1 0]
 [0 2 0]
 [3 0 2]]


In [22]:
cm_l2 = metrics.confusion_matrix(y_test, predictions_l2)
print(cm_l2)

[[0 1 0]
 [0 2 0]
 [3 0 2]]


## Probabilities

In [23]:
clf_l1_LR.predict_proba(x_test)

array([[0.20089303, 0.48165593, 0.31745104],
       [0.39587385, 0.13397951, 0.47014665],
       [0.40647729, 0.14126043, 0.45226228],
       [0.04293027, 0.86843974, 0.08863   ],
       [0.17791383, 0.52891658, 0.29316959],
       [0.65794876, 0.16415683, 0.17789441],
       [0.4043328 , 0.30162753, 0.29403967],
       [0.73313554, 0.10622685, 0.1606376 ]])

## Informations about the models

In [24]:
coef_l1_LR = clf_l1_LR.coef_.ravel()
coef_l2_LR = clf_l2_LR.coef_.ravel()

# coef_l1_LR contains zeros due to the
# L1 sparsity inducing norm

sparsity_l1_LR = np.mean(coef_l1_LR == 0) * 100
sparsity_l2_LR = np.mean(coef_l2_LR == 0) * 100

In [25]:
print("Number of parameters", len(coef_l1_LR))
print("Sparcity", sparsity_l1_LR)

Number of parameters 45312
Sparcity 92.54943502824858


In [26]:
print(sparsity_l2_LR)

0.0


## Relevant features

In [27]:
print(clf_l1_LR.coef_.shape)

(3, 15104)


In [28]:
categories = {}

for i,cat_features in enumerate(clf_l1_LR.coef_):
    relevant_features = np.where(cat_features != 0)
    categories[i] = df_features.columns[relevant_features].to_list()
    print("Number of relevant features for category {} :".format(i),len(relevant_features[0]))
    

Number of relevant features for category 0 : 1190
Number of relevant features for category 1 : 1197
Number of relevant features for category 2 : 989


In [29]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

In [30]:
intersection_cat0_cat1 = intersection(categories[0], categories[1])
print("Common features between cat0 and cat1", len(intersection_cat0_cat1))

Common features between cat0 and cat1 531


In [31]:
intersection_cat1_cat2 = intersection(categories[1], categories[2])
print("Common features between cat1 and cat2", len(intersection_cat1_cat2))

Common features between cat1 and cat2 337


In [32]:
intersection_cat2_cat0 = intersection(categories[2], categories[0])
print("Common features between cat2 and cat0", len(intersection_cat2_cat0))

Common features between cat2 and cat0 341


In [33]:
from collections import Counter
from collections import defaultdict

def get_uncommon_features(categories):
    n = len(categories.keys())
    intersection  = []
    unique_features = {}
    
    for k in range(n):
        unique_features[k] = Counter(categories[k])
    
    for i in range(n):
        features_cat0 = Counter(categories[i%3])
        features_cat1 = Counter(categories[(i+1)%3])
        intersection = features_cat0 & features_cat1
        unique_features[i] = unique_features[i] - intersection
        unique_features[(i+1)%3] = unique_features[(i+1)%3] - intersection
    
    for k in range(n):
        unique_features[k] = list(unique_features[k].elements())
    
    return unique_features

In [34]:
unique_features = get_uncommon_features(categories)

In [35]:
print("Unique features for category 0:", len(unique_features[0]))

Unique features for category 0: 416


In [36]:
print("Unique features for category 1:", len(unique_features[1]))

Unique features for category 1: 427


In [37]:
print("Unique features for category 2:", (len(unique_features[2])))

Unique features for category 2: 409


# Logisitc regression with selected features

In [38]:
selected_data = pd.read_csv("../data/selected_train.csv", sep=",")

In [39]:
df_features_selected = selected_data[selected_data.columns.difference(['label'])]
df_labels_selected = selected_data["label"]

In [40]:
X_selected = df_features_selected.values
print(X_selected.shape)

y_selected = df_labels.values
print(y_selected.shape)

(29, 56)
(29,)


In [41]:
from sklearn.model_selection import train_test_split
x_train_s, x_test_s, y_train_s, y_test_s = train_test_split(X_selected, y_selected, test_size=0.6, random_state=0)

In [42]:
clf_l2_selected = LogisticRegression(penalty='l2', solver='saga', max_iter=1000, multi_class='multinomial', C=0.01)

In [43]:
clf_l2_selected.fit(x_train_s, y_train_s)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
predict = clf_l2_selected.predict(x_test_s)

In [45]:
score_l2_s = clf_l2_selected.score(x_test_s, y_test_s)
print(score_l2_s)

0.8333333333333334


In [46]:
cm_l2_s = metrics.confusion_matrix(y_test_s, predict)
print(cm_l2_s)

[[5 0 0]
 [0 6 0]
 [2 1 4]]
